# QuHack Submission

The idea is to create a proof of concept of entanglement for as many qubits as possible

# Theortical Tests

The [Woronowicz–Horodecki criterion and Psotive Partial tranpose](https://cs.uwaterloo.ca/~watrous/TQI-notes/TQI-notes.18.pdf) are mathematical theory useful for proving seperability and entanglement

We will put quantum circuits to 3 different tesr to qualify entanglement
1. Positive Partial Tranpose Test (2-qubit systems Only)
   * Entamgled if the minimum eigenvalue is < 0
2. [Negativity Tests](https://en.wikipedia.org/wiki/Peres%E2%80%93Horodecki_criterion)
    * Entangled if $\sum(|\lambda_i|) < 0$
3. [Von Neuman Entropy Test](https://www.scottaaronson.com/qclec/11.pdf) (For pure states)
     * Entangled if $-Tr(\rho \ln(\rho))$ $-\sum(\lambda_i \log(\lambda_i))_i$> 0

In [1]:
import numpy as np
import random

import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import DensityMatrix, partial_trace, entropy

C:\Users\meep_\AppData\Local\Programs\Python\Python311\Lib\site-packages\qiskit\circuit\library\pauli_evolution.py:19: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.0)
  import scipy as sc


In [2]:
class Entanglement_Tests:
    def partial_transpose(self, rho, dims, sys =1):
        """
        Perform a partial transpose on a given matrix. Default is set to T_B
        """
        dA, dB = dims
        rho_reshaped = rho.copy()
        rho_reshaped = rho_reshaped .reshape(dA, dB, dA, dB)
    
        # Tranpose subsystem A
        if sys == 0:
            rho_pt = rho_reshaped.transpose(2,1,0,3)
        # Transpose subsytem B
        else:
            rho_pt = rho_reshaped.transpose(0,3,2,1)
    
        return rho_pt.reshape(dA*dB, dA*dB)

    def ppt_criterion(self, rho):
        """
        PPT (Positive Partial Transpose) test
        If partial transpose has negative eigenvalues, state is for 2 qubit systems
        """
        dims = (2,2) # Only run on a 2 qubit system
        rho_pt=  self.partial_transpose(rho, dims)
        
        eigenvalues = np.linalg.eigvalsh(rho_pt)
        min_eigenvalue = np.min(eigenvalues)
        
        is_entangled = min_eigenvalue < -1e-10
        return is_entangled, min_eigenvalue

    def negativity_test(self, rho, num_qubits):
        """
        Negativity: sum of absolute values of negative eigenvalues of partial transpose
        N > 0 proves entanglement
        """
        # Obtain dims from qubits
        dims = (2**(num_qubits -1) , 2)
        # Partial transpose all 
        rho_pt=  self.partial_transpose(rho, dims)
        
        eigenvalues = np.linalg.eigvalsh(rho_pt)
        negativity = np.sum(np.abs(eigenvalues[eigenvalues < 0]))
        return negativity

    def von_neumann_entropy_criterion(self, dm, n_qubits):
        """
        For pure states: if reduced density matrix has S > 0, state is entangled
        """
       # Check if state is pure (Tr(rho^2) ≈ 1)
        purity = np.real(np.trace(dm.data @ dm.data))
        
        if purity < 0.99:
            return None, "State not pure enough for this test"
        
        # Trace out all but first qubit
        reduced_dm = partial_trace(dm, list(range(1, n_qubits)))
        
        entropy_val = entropy(reduced_dm)
        
        is_entangled = entropy_val > 1e-5  # Small threshold for numerical errors
        return is_entangled, entropy_val

    def test(self, states: dict):
        """
        Run all 3 Tests and output print results in a meaningful way

        Args:
        ::states:: A Dictionary of quantum circuits with keys named after the resulting state
        """

        print("="* 80)
        print("ENTANGLEMENT TESTS")
        print("="* 80)

        for name, qc in states.items():
            print(f"\n{name}:")
            print(qc.draw(output='text', fold = -1))
            
            dm = DensityMatrix(qc)
            rho = dm.data
            num_qubits = qc.num_qubits
            
            # Test 1: PPT Criterion (DEFINITIVE for 2 qubits)
            if num_qubits == 2:
                is_ent_ppt, min_eig = self.ppt_criterion(rho)
                print(f"  PPT Test: {'✓ ENTANGLED' if is_ent_ppt else '✗ Separable'}")
                print(f"    Min eigenvalue of ρ^T_B: {min_eig:.6f}")
            
            # Test 2: Negativity
            neg = self.negativity_test(rho, num_qubits)
            print(f"  Negativity: {neg:.6f} {'✓ ENTANGLED' if neg > 1e-3 else '✗ Separable'}")
            
            # Test 3: von Neumann Entropy
            is_ent_vn, entropy_val = self.von_neumann_entropy_criterion(dm, num_qubits)
            if is_ent_vn is not None:
                print(f"  Von Neumann Entropy: {entropy_val:.6f} {'✓ ENTANGLED' if is_ent_vn else '✗ Separable'}")
            
    
    

# Unentangled States (Control)

In [3]:
unentangled_states = {}
# state1
state1 = QuantumCircuit(2)
state1.x(0)
state1.cx(0,1)
state1.ry(np.pi,1)
unentangled_states["2_qubit+state"] = state1
#state 2
state2 = QuantumCircuit(4)
state2.y(0)
state2.cx(0,2)
state2.cx(2,0)
state2.x(0)
unentangled_states["4_qubit+state"] = state2
#State3
state3 = QuantumCircuit(7)
for i in range(7):
    if i <4:
        state3.x(i)
        state3.cx(i, i+1)
    state3.x(i)
    
unentangled_states["7_qubit+state"] = state3

et = Entanglement_Tests() 
et.test(unentangled_states)

ENTANGLEMENT TESTS

2_qubit+state:
     ┌───┐              
q_0: ┤ X ├──■───────────
     └───┘┌─┴─┐┌───────┐
q_1: ─────┤ X ├┤ Ry(π) ├
          └───┘└───────┘
  PPT Test: ✗ Separable
    Min eigenvalue of ρ^T_B: 0.000000
  Negativity: 0.000000 ✗ Separable
  Von Neumann Entropy: 0.000000 ✗ Separable

4_qubit+state:
     ┌───┐     ┌───┐┌───┐
q_0: ┤ Y ├──■──┤ X ├┤ X ├
     └───┘  │  └─┬─┘└───┘
q_1: ───────┼────┼───────
          ┌─┴─┐  │       
q_2: ─────┤ X ├──■───────
          └───┘          
q_3: ────────────────────
                         
  Negativity: 0.000000 ✗ Separable
  Von Neumann Entropy: 0.000000 ✗ Separable

7_qubit+state:
     ┌───┐     ┌───┐                              
q_0: ┤ X ├──■──┤ X ├──────────────────────────────
     └───┘┌─┴─┐├───┤     ┌───┐                    
q_1: ─────┤ X ├┤ X ├──■──┤ X ├────────────────────
          └───┘└───┘┌─┴─┐├───┤     ┌───┐          
q_2: ───────────────┤ X ├┤ X ├──■──┤ X ├──────────
                    └───┘└───┘┌─┴─┐├───┤     ┌──

# Bell States

In [4]:
def create_bell_states():
    """4 maximally entangled Bell states"""
    states = {}
    
    # |Φ+⟩ = (|00⟩ + |11⟩)/√2
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    states['Bell Φ+'] = qc
    
    # |Φ-⟩ = (|00⟩ - |11⟩)/√2
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    qc.z(0)
    states['Bell Φ-'] = qc
    
    # |Ψ+⟩ = (|01⟩ + |10⟩)/√2
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    qc.x(1)
    states['Bell Ψ+'] = qc
    
    # |Ψ-⟩ = (|01⟩ - |10⟩)/√2
    qc = QuantumCircuit(2)
    qc.x(0)
    qc.h(0)
    qc.cx(0, 1)
    states['Bell Ψ-'] = qc
    
    return states

bell_states = create_bell_states()
et.test(bell_states)

ENTANGLEMENT TESTS

Bell Φ+:
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
  PPT Test: ✓ ENTANGLED
    Min eigenvalue of ρ^T_B: -0.500000
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy: 1.000000 ✓ ENTANGLED

Bell Φ-:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ Z ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├─────
          └───┘     
  PPT Test: ✓ ENTANGLED
    Min eigenvalue of ρ^T_B: -0.500000
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy: 1.000000 ✓ ENTANGLED

Bell Ψ+:
     ┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐┌───┐
q_1: ─────┤ X ├┤ X ├
          └───┘└───┘
  PPT Test: ✓ ENTANGLED
    Min eigenvalue of ρ^T_B: -0.500000
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy: 1.000000 ✓ ENTANGLED

Bell Ψ-:
     ┌───┐┌───┐     
q_0: ┤ X ├┤ H ├──■──
     └───┘└───┘┌─┴─┐
q_1: ──────────┤ X ├
               └───┘
  PPT Test: ✓ ENTANGLED
    Min eigenvalue of ρ^T_B: -0.500000
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy:

# GHZ States

In [5]:
def create_ghz_states(qubits= [3,4,5,6,7,8]):
    """
    Create a dictionary of GHZ states from a list of ints
    
    GHZ state: (|000...0⟩ + |111...1⟩)/√2 with 
    """
    assert all(n > 2 for n in qubits), "Cannot make state out of less than 3 qubits"

    states = {}
    for n in qubits:
        qc = QuantumCircuit(n)
        qc.h(0)
        for i in range(1, n):
            qc.cx(0, i)
        # Add to state dict
        key = "GHZ-" + str(n)
        states[key] = qc

    return states
ghz = create_ghz_states()
et.test(ghz)


ENTANGLEMENT TESTS

GHZ-3:
     ┌───┐          
q_0: ┤ H ├──■────■──
     └───┘┌─┴─┐  │  
q_1: ─────┤ X ├──┼──
          └───┘┌─┴─┐
q_2: ──────────┤ X ├
               └───┘
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy: 1.000000 ✓ ENTANGLED

GHZ-4:
     ┌───┐               
q_0: ┤ H ├──■────■────■──
     └───┘┌─┴─┐  │    │  
q_1: ─────┤ X ├──┼────┼──
          └───┘┌─┴─┐  │  
q_2: ──────────┤ X ├──┼──
               └───┘┌─┴─┐
q_3: ───────────────┤ X ├
                    └───┘
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy: 1.000000 ✓ ENTANGLED

GHZ-5:
     ┌───┐                    
q_0: ┤ H ├──■────■────■────■──
     └───┘┌─┴─┐  │    │    │  
q_1: ─────┤ X ├──┼────┼────┼──
          └───┘┌─┴─┐  │    │  
q_2: ──────────┤ X ├──┼────┼──
               └───┘┌─┴─┐  │  
q_3: ───────────────┤ X ├──┼──
                    └───┘┌─┴─┐
q_4: ────────────────────┤ X ├
                         └───┘
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy: 1.000000 ✓ ENTANGLE

# Cluster States

In [6]:
def create_cluster_states(qubits = [3,4,5,6,7]):
    """ 
    Create a dictionary of 1D Cluster states from a list of ints
    """
    
    assert all(n > 2 for n in qubits), "Cannot make state out of less than 3 qubits"
    states = {}
    for n in qubits:
        qc = QuantumCircuit(n)
        # Initialize in |+⟩ states
        for i in range(n):
            qc.h(i)
        # Apply CZ gates between neighbors
        for i in range(n - 1):
            qc.cz(i, i + 1)

        key = "Cluster " + str(n) + " qubits"
        states[key] = qc
        
    return states

cluster = create_cluster_states()
et.test(cluster)

ENTANGLEMENT TESTS

Cluster 3 qubits:
     ┌───┐      
q_0: ┤ H ├─■────
     ├───┤ │    
q_1: ┤ H ├─■──■─
     ├───┤    │ 
q_2: ┤ H ├────■─
     └───┘      
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy: 1.000000 ✓ ENTANGLED

Cluster 4 qubits:
     ┌───┐         
q_0: ┤ H ├─■───────
     ├───┤ │       
q_1: ┤ H ├─■──■────
     ├───┤    │    
q_2: ┤ H ├────■──■─
     ├───┤       │ 
q_3: ┤ H ├───────■─
     └───┘         
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy: 1.000000 ✓ ENTANGLED

Cluster 5 qubits:
     ┌───┐            
q_0: ┤ H ├─■──────────
     ├───┤ │          
q_1: ┤ H ├─■──■───────
     ├───┤    │       
q_2: ┤ H ├────■──■────
     ├───┤       │    
q_3: ┤ H ├───────■──■─
     ├───┤          │ 
q_4: ┤ H ├──────────■─
     └───┘            
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy: 1.000000 ✓ ENTANGLED

Cluster 6 qubits:
     ┌───┐               
q_0: ┤ H ├─■─────────────
     ├───┤ │             
q_1: ┤ H ├─■──■──────────
     ├───┤  

# W States

Note that W states for more than 3 qubits [is mostly a generalization](https://en.wikipedia.org/wiki/W_state)

In [7]:
def create_w_states(qubits = [3,4,5,6,7,8]):
    """
    Creat a dictionary of W states out of list of ints
    W state: (|100...0⟩ + |010...0⟩ + ... + |000...1⟩)/√n
    """
    assert all(n > 2 for n in qubits), "Cannot make state out of less than 3 qubits"
    
    states = {}
    for n in qubits:
        if n == 3:
            qc = QuantumCircuit(3)
            # Prepare |W⟩ = (|100⟩ + |010⟩ + |001⟩)/√3
            angle = 2 * np.arccos(np.sqrt(1/3))
            qc.ry(angle, 0)
            qc.ch(0, 1)
            qc.cx(1, 0)
            qc.x(0)
            qc.ccx(0, 1, 2)
            qc.x(0)
        else:
            # Approximate W state for other sizes
            qc = QuantumCircuit(n)
            qc.h(0)
            for i in range(1, n):
                qc.cx(0, i)
            # Add some variation
            qc.ry(np.pi/4, n//2)

        key = "W-" + str(n)
        states[key] = qc
    
    return states

w_states = create_w_states()
et.test(w_states)

ENTANGLEMENT TESTS

W-3:
     ┌────────────┐     ┌───┐┌───┐     ┌───┐
q_0: ┤ Ry(1.9106) ├──■──┤ X ├┤ X ├──■──┤ X ├
     └────────────┘┌─┴─┐└─┬─┘└───┘  │  └───┘
q_1: ──────────────┤ H ├──■─────────■───────
                   └───┘          ┌─┴─┐     
q_2: ─────────────────────────────┤ X ├─────
                                  └───┘     
  Negativity: 0.333333 ✓ ENTANGLED
  Von Neumann Entropy: 0.550048 ✓ ENTANGLED

W-4:
     ┌───┐                          
q_0: ┤ H ├──■────■────■─────────────
     └───┘┌─┴─┐  │    │             
q_1: ─────┤ X ├──┼────┼─────────────
          └───┘┌─┴─┐  │  ┌─────────┐
q_2: ──────────┤ X ├──┼──┤ Ry(π/4) ├
               └───┘┌─┴─┐└─────────┘
q_3: ───────────────┤ X ├───────────
                    └───┘           
  Negativity: 0.500000 ✓ ENTANGLED
  Von Neumann Entropy: 1.000000 ✓ ENTANGLED

W-5:
     ┌───┐                               
q_0: ┤ H ├──■────■────■───────────────■──
     └───┘┌─┴─┐  │    │               │  
q_1: ─────┤ X ├──┼────┼────────

# Random Entangled States
Ultimately, this may or may not be where my implmentatiun falls apart, depending on the seed. I doubt that I accidentally made non-entangled states, but I encourage people to run this cell multiple times without settinng a seed to see what happens 

In [8]:
def create_random_entangled(qubits_range = [2,3,4,5,6,7], circuits = 3, depths=[2,3], seed = None):
    """
    Create a dictionary of randomly generated entangled circuits 
    from a list of intas
    """
    states = {}
    
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
        
    # Create number of circuits
    for n in range(circuits):
        # Pick a range of qubits
        num_qubits = random.choice(qubits_range)
        
        qc = QuantumCircuit(num_qubits)
        # Pick a random depth
        depth = random.choice(depths)
        
        for __ in range(depth):
            # Random single-qubit rotations
            for i in range(num_qubits):
                qc.rx(np.random.uniform(0, 2*np.pi), i)
                qc.rz(np.random.uniform(0, 2*np.pi), i)
            
            # Random CNOT gates
            for i in range(num_qubits - 1):
                if np.random.random() > 0.5:
                    qc.cx(i, i + 1)
    
        key = "Random Circuit " + str(n +1)
        states[key] = qc
        
    return states

random_states = create_random_entangled()
et.test(random_states)

ENTANGLEMENT TESTS

Random Circuit 1:
     ┌────────────┐ ┌───────────┐ ┌────────────┐┌────────────┐ ┌────────────┐┌─────────────┐               
q_0: ┤ Rx(5.0452) ├─┤ Rz(5.911) ├─┤ Rx(4.4227) ├┤ Rz(4.6594) ├─┤ Rx(5.6492) ├┤ Rz(0.62642) ├───────■───────
     ├────────────┤┌┴───────────┴┐├────────────┤├────────────┤┌┴────────────┤└┬────────────┤     ┌─┴─┐     
q_1: ┤ Rx(2.6917) ├┤ Rz(0.56831) ├┤ Rx(6.2662) ├┤ Rz(4.6415) ├┤ Rx(0.11874) ├─┤ Rz(3.5606) ├─────┤ X ├─────
     ├────────────┤└┬────────────┤└────────────┘├────────────┤└┬────────────┤ ├────────────┤┌────┴───┴────┐
q_2: ┤ Rx(3.7969) ├─┤ Rz(5.4393) ├──────■───────┤ Rx(5.5327) ├─┤ Rz(3.9404) ├─┤ Rx(3.2504) ├┤ Rz(0.73127) ├
     ├───────────┬┘┌┴────────────┤    ┌─┴─┐     ├────────────┤ ├────────────┤ ├────────────┤└┬────────────┤
q_3: ┤ Rx(6.283) ├─┤ Rz(0.58268) ├────┤ X ├─────┤ Rx(2.3463) ├─┤ Rz(4.4376) ├─┤ Rx(1.9153) ├─┤ Rz(1.4986) ├
     └───────────┘ └─────────────┘    └───┘     └────────────┘ └────────────┘ └────────────┘ └────

# IQM Tests

In [9]:
from iqm.qiskit_iqm import IQMProvider

token = "Yoken here" # Change accordingly

provider = IQMProvider("https://resonance.meetiqm.com",
                       quantum_computer="garnet",
                       token=input(token))
backend = provider.get_backend()


results=job.result()
print(results.get_counts())


def run_iqm(backend, qc_dict, save_output = False):
    """
    Take a dictionary of qunatum states and run them on a IQM quantum computer
    """

    # List of circuits
    circuits = []
    for qc in qc_dict.values():
        qc_transpiled = transpile(qc,backend) # Get the circuit ready for IQM hardware
        circuits.append(qc_transpiled)
        
    state_names = list(qc_dict.keys())

    # Run jobs
    jobs = backend.run(circuit_list)
    counts = jobs.result().get_counts()
    job_id = jobs.job_id()  

    # Plot Counts Dict with Job
    for i in range(len(counts)):
        plt.figure()
        plt.title("Results of " + state_names[i])
        plt.bar(counts[i].keys(), counts[i].values(), color='g')
        plt.show()
        
    # Return if needed
    if save_output:
        return job_id, counts, state_names


ModuleNotFoundError: No module named 'iqm'

In [ ]:
run_iqm(backend,unentangled_states)

In [ ]:
run_iqm(backend, bell_states)

In [ ]:
run_iqm(backend, ghz)

In [ ]:
run_iqm(backend, cluster)

In [ ]:
run_iqm(backend, w_states)

In [ ]:
run_iqm(backend, random_states)